In [27]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy import interpolate
import importlib.resources as pkg_resources
from pistachio import transfer_matrix as tm
from pistachio import default

## Build a structure from a pre-populated yaml config file

The `load_struct_from_config` method uses a yaml config file created by you to 
generate a multi-layered structure. In our example, there are two CaF2 layers and two
Au layers. The data for these layers are downloaded from refractiveindex.info, so
the number of data points between them differs. The middle layer is air, which just
has a single refractive index for all wavelengths provided. `load_struct_from_config` will
refactor each of these layers so that they have the same set of wavelengths so that
the transfer matrix can be calculated for each wavelength between the user-defined
max and min wavelengths.

Each layer is a class that you can access and modify.
A structure is essentially a list of layer classes with methods that can act on
all layers, such as performing layer transfer matrices and calculating the total transmittance
and reflectance for light propagating through the entire structure.

Here, we demonstrate calculating the transmittance and reflectance of light travelling 
through the entire structure.

In [28]:
yaml_file = 'fabry-perot.yaml'
with pkg_resources.path(default, yaml_file) as yml:
    yaml_config = os.path.abspath(yml)

s = tm.Structure()
s.load_struct_from_config(yaml_config)

Initializing structure...


In [33]:
# Print the parameters of our structure.
s.print_structure()

Structure Configuration
Layer 0 : CaF2 | d = 0 nm
Layer 1 : Au | d = 20 nm
Layer 2 : Air | d = 10000 nm
Layer 3 : Au | d = 20 nm
Layer 4 : CaF2 | d = 0 nm


## Calculate Transfer Matrix, Transmittance, and Reflectance

Set the angle of incidence (zero degrees) and light polarization (s-wave). Then 
we calculate each layer transfer matrix at every wavelength and then generate the total
structure transfer matrix (again at each wavelength). A list of transfer matrices 
is returned. Then `calculate_all_t_r` calculates the transmittance and reflectance
from each provided total transfer matrix. From this we can plot the transmittance and 
reflectance spectrum for our structure.

In [34]:
theta = 0.
polarization = 's-wave'
M_all = s.calculate_all_transfer_matrices(theta, polarization)
T_all, R_all = s.calculate_all_t_r(M_all)

In [35]:
# Convert to wavenumbers (cm-1) and load everything we calculated into a Pandas dataframe.
# This is just for convenience when using Plotly to graph our data.

wavenumbers = (1 / s.wavelengths) / 100
T_percent = T_all * 100
df = pd.DataFrame({"Wavenumber (cm-1)": wavenumbers, "Transmittance (%)": T_percent, "Reflectance (%)": R_all})

In [44]:
fig = px.line(df, x='Wavenumber (cm-1)', y='Transmittance (%)', title='Fabry-Pérot etalon of distance {} μm'.format(s.layers[2].thickness*10**6))
fig.show()